## Aram Data and Analysis

In [52]:
# imports
from league_data import *
from league_functions import *
from league_constants import *
import cassiopeia as cass
import plotly.express as px
import seaborn as sns

#### TODO
-convert dill dumping to csv
<br> -finish fuctions
<br> -new function ideas


#### Load Data

In [65]:
%%time
df = load_fetch_data(zeb)

DATA LOADED 23
NO DATA ADDED

CPU times: user 7.2 ms, sys: 2.43 ms, total: 9.63 ms
Wall time: 7.9 ms


In [54]:
champs = pd.Series(cass.get_champions(region="NA"))
champs.index = champs.apply(lambda x: x.name)
champs = champs.apply(lambda x: x.tags[0])

adc = set(champs[champs.apply(lambda x: "Marksman" in x)].index)
assassin = set(champs[champs.apply(lambda x: "Assassin" in x)].index)
fighter = set(champs[champs.apply(lambda x: "Fighter" in x)].index)
support = set(champs[champs.apply(lambda x: "Support" in x)].index)
tank = set(champs[champs.apply(lambda x: "Tank" in x)].index)
mage = set(champs[champs.apply(lambda x: "Mage" in x)].index)


#### Clean data

In [55]:
df.duration = df.duration.astype('timedelta64[s]')
df.id = pd.to_numeric(df.id)
df.patch = pd.to_numeric(df.patch.apply(lambda x: x.name))
df.creation = pd.to_datetime(df.creation.apply(lambda x:str(x.utcnow())))
df.surrender = df.surrender.astype('bool')
df["side"] = df.where(df.side == 1).red.fillna(df.where(df.side == 0).blue)
df["teammates"] = df.side.apply(lambda x: {i.name for i in x})
df["teamcomp"] = df.side.apply(lambda x: {i.champ for i in x})
df["win"] = df.summoner.apply(lambda x: x.stats.win)

df.head(1)

,duration,patch,id,creation,surrender,blue,red,summoner,side,teammates,teamcomp,win
0,1315.0,11.12,3943927493,2021-07-13 02:52:46.414068+00:00,True,"[(Derroz, Viktor), (Yze, Jarvan IV), (Dodge Do...","[(DrtyPirateHooker, Zed), (ItzFiþ, Blitzcrank)...",summoner_spell_d: Flash\nsummoner_spell_f: Gho...,"[(Derroz, Viktor), (Yze, Jarvan IV), (Dodge Do...","{Yze, rdinho9, Big Benis Boy, Dodge Down, Derroz}","{Amumu, Jarvan IV, Caitlyn, Maokai, Viktor}",True


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   duration   23 non-null     float64            
 1   patch      23 non-null     float64            
 2   id         23 non-null     int64              
 3   creation   23 non-null     datetime64[ns, UTC]
 4   surrender  23 non-null     bool               
 5   blue       23 non-null     object             
 6   red        23 non-null     object             
 7   summoner   23 non-null     object             
 8   side       23 non-null     object             
 9   teammates  23 non-null     object             
 10  teamcomp   23 non-null     object             
 11  win        23 non-null     bool               
dtypes: bool(2), datetime64[ns, UTC](1), float64(2), int64(1), object(6)
memory usage: 2.0+ KB


#### Filter

In [57]:
df = df[df.patch.apply(round) == 11]
len(df)

11

In [58]:
solo = df[df.teammates.apply(lambda x: len(x & friends) == 0)]
premade = df[df.teammates.apply(lambda x: len(x & friends) != 0)]

In [59]:
wr(df)

Win     54.5% 6
Loss    45.5% 5
Name: win, dtype: object

In [60]:
wr(solo)

KeyError: 'Win'

In [ ]:
wr(premade)
# df = solo

In [ ]:
wl = pd.DataFrame()

wl["wins"] = df[df.win == True].summoner.apply(lambda x: x.champion).value_counts()
wl["losses"] = df[df.win == False].summoner.apply(lambda x: x.champion).value_counts()
wl["losses"].fillna(value=0, inplace=True)
wl["total"] = wl.losses + wl.wins
wl["wr"] = round(wl.wins / wl.total, 2) * 100

wl[wl.total > 3].sort_values(by=["wr", "total"], ascending=False)

In [ ]:
adc_games = df[df.teamcomp.apply(lambda x: len(x & adc) != 0)]
no_adc_games = df[df.teamcomp.apply(lambda x: len(x & adc) == 0)]

In [ ]:
print(len(adc_games))
wr(adc_games)

In [ ]:
print(len(no_adc_games))
wr(no_adc_games)

In [ ]:
sns.histplot(df[["duration","win"]], x="duration", hue="win", element="step",kde=True)


#### Win Time distribution

In [ ]:
df[df.win == True].duration.describe().drop("count").apply(lambda x: str(int(x // 60))+"m "+str(int(x%60))+"s")

#### Loss Time distribution

In [ ]:
df[df.win == False].duration.describe().drop("count").apply(lambda x: str(int(x // 60))+"m "+str(int(x%60))+"s")

In [ ]:
from collections import Counter

Counter(df.teammates.apply(list).sum()).most_common()[:50]

In [ ]:
wr(df[df.teamcomp.apply(lambda x: "Thresh" in x)])

In [ ]:
# win_rate(data)
# champ_winrate(data)
# champ_kda(robb, Aram_Data)
# kda(Aram_Data)
# kp(Aram_Data)
# common_teammates(Aram_Data)
# multikills(Aram_Data)
# items(Aram_Data)
# adc_winrate(data)
# time_played(Aram_Data)
# win_rate_with(thomas, Aram_Data)
# time_distribution(robb, data)

wr(df[df.side.apply(lambda x: x[0].champ in adcs)])

In [ ]:
wr(df[df.side.apply(lambda x: x[0].champ in assassin)])

In [ ]:
wr(df[df.side.apply(lambda x: x[0].champ in fighter)])

In [ ]:
wr(df[df.side.apply(lambda x: x[0].champ in support)])

In [ ]:
wr(df[df.side.apply(lambda x: x[0].champ in tank)])

In [ ]:
wr(df[df.side.apply(lambda x: x[0].champ in mage)])
# a  = dill.load(open("Aram_Data_New.pickle", "rb"))
# print(a[0].summoner.stats.first_inhibitor_kill)